# 布林拐头策略

In [2]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime as dt
from itertools import product
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import time

# 回测框架
import backtrader as bt
import backtrader.indicators as btind
import backtrader.feeds as btfeed

# 基础函数
import utilsJ

## 策略主体

买入信号：波浪均线出现向上拐头（前两日均下降，今日上升）且当日收盘价在布林带中线之上。

买入仓位：满仓买入。

卖出信号：波浪均线出现向下拐头（前两日均上升，今日下降）或收盘价向下突破布林带上界。

卖出仓位：一律平仓。

注：该策略主要针对股票，因此未考虑做空情况。

### 个股版本

In [3]:
class BW_s(bt.Strategy):
    
    params = (
        # General params
        ('printlog', False),
        ('units', 1),
        ('stake', 100),

        # Indicator params
        ('bbands_per', 20),
        ('bbands_dev', 2),
        ('direction_lag', 5),
        ('wave_period', 7),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        if self.p.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):

        # Initialization
        self.buyprice = 0
        self.sellprice = 0
        self.order = None

        # Alias
        self.dataclose = self.datas[0].close
        self.datahigh = self.datas[0].high
        self.datalow = self.datas[0].low

        # Indicators
        ## Bollinger
        self.bbands = btind.BBands(self.dataclose, 
                                   period=self.p.bbands_per, 
                                   devfactor=self.p.bbands_dev)
        self.bbands_sell = bt.And(self.dataclose < self.bbands.top,
                                  self.dataclose(-1) > self.bbands.top(-1))

        ## Wave
        self.sma = btind.SMA(self.dataclose, period=self.p.wave_period)
        self.wave_buy = bt.And(self.sma(0) > self.sma(-1), self.sma(-1) < self.sma(-2), 
                               self.sma(-2) < self.sma(-3), self.sma(0) > self.bbands.mid(0))
        self.wave_sell = bt.And(self.sma(0) < self.sma(-1), self.sma(-1) > self.sma(-2), 
                                self.sma(-2) > self.sma(-3))


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price: %.2f, Lot:%i, Position:%i, Cash: %i, Value: %i' %
                         (order.executed.price, order.executed.size,
                          self.getposition(self.data).size,
                          self.broker.get_cash(), self.broker.get_value()))
                self.buyprice = order.executed.price
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Lot:%i, Position:%i, Cash: %i, Value: %i' %
                         (order.executed.price, -order.executed.size,
                          self.getposition(self.data).size,
                          self.broker.get_cash(), self.broker.get_value()))
                self.sellprice = order.executed.price

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
        
        self.order = None


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))


    def next(self):
        
        if self.order:
            return
        
        buy_s = self.wave_buy[0]
        sell_s = self.bbands_sell[0] or self.wave_sell[0]
        
        if buy_s and self.getposition(self.data).size == 0: # Open position
            ava_pos = ((self.broker.get_cash() / self.p.units) // (self.dataclose[0]*self.p.stake)) * self.p.stake
            if ava_pos > 0:
                self.log('BUY CREATE, Price: %.2f, Lots: %i' % 
                         (self.dataclose[0], ava_pos))
                self.order = self.buy(size=ava_pos)
                
        elif sell_s and self.getposition(self.data).size > 0: # Close position
            self.log('Sell CREATE(Close), Price: %.2f, Lots: %i' % 
                     (self.dataclose[0], self.getposition(self.data).size))
            self.order = self.close()

    
    def stop(self):
        self.log('Ending Position:%i. Ending Value:%.2f.' %
                 (self.getposition(self.data).size,
                 self.cerebro.broker.getvalue()), doprint=True)

### 多股版本

In [16]:
class BW_m(bt.Strategy):
    
    params = (
        # General params
        ('printlog', False),
        ('units', 300),
        ('stake', 100),

        # Indicator params
        ('bbands_per', 20),
        ('bbands_dev', 2),
        ('wave_period_buy', 7),
        ('wave_period_sell', 7),
    )

    
    def log(self, txt, dt=None, doprint=False):
        if self.p.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))

    
    def __init__(self):

        # Global initialization
        self.inds = dict()
        self.cash_unit = self.broker.get_cash() / self.p.units
        self.tracker = None

        for d in self.datas:
            # Local initialization
            self.inds[d] = dict()
            self.inds[d]['buyprice'] = 0
            self.inds[d]['sellprice'] = 0
            self.inds[d]['order'] = None

            # Indicators
            ## Bollinger Bands
            self.inds[d]['bbands'] = btind.BBands(d.close, 
                                                  period=self.p.bollinger_per, 
                                                  devfactor=self.p.bollinger_dev)
            self.inds[d]['bbands_buy'] = bt.And(d.close(0) > self.inds[d]['bbands'].mid(0), 
                                                d.close(-1) < self.inds[d]['bbands'].mid(-1))

            self.inds[d]['bbands_sell'] = bt.And(d.close(0) < self.inds[d]['bbands'].top(0), 
                                                 d.close(-1) > self.inds[d]['bbands'].top(-1))

            ## Wave
            self.inds[d]['sma_b'] = btind.SMA(d.close, period=self.p.wave_period_buy)
            self.inds[d]['sma_s'] = btind.SMA(d.close, period=self.p.wave_period_sell)
            self.inds[d]['wave_buy'] = bt.And(self.inds[d]['sma_b'](0) > self.inds[d]['sma_b'](-1), 
                                              self.inds[d]['sma_b'](-1) < self.inds[d]['sma_b'](-2), 
                                              self.inds[d]['sma_b'](-2) < self.inds[d]['sma_b'](-3),
                                              self.inds[d]['sma_b'](0) > self.inds[d]['bbands'].mid(0))

            self.inds[d]['wave_sell'] = bt.And(self.inds[d]['sma_s'](0) < self.inds[d]['sma_s'](-1), 
                                               self.inds[d]['sma_s'](-1) > self.inds[d]['sma_s'](-2), 
                                               self.inds[d]['sma_s'](-2) > self.inds[d]['sma_s'](-3))


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('买单执行, 代码：%s, 价格:%.2f, 股数:%i, 持有现金:%i' %
                         (order.info['name'], order.executed.price,
                          order.executed.size, self.broker.get_cash()))
                self.inds[self.tracker]['buyprice'] = order.executed.price

            else:  # Sell
                self.log('卖单执行, 代码：%s, 价格:%.2f, 股数:%i, 持有现金:%i' %
                        (order.info['name'], order.executed.price,
                         -order.executed.size, self.broker.get_cash()))
                self.inds[self.tracker]['sellprice'] = order.executed.price

        elif order.status in [order.Canceled]:
            self.log('订单取消：被撤销')

        elif order.status in [order.Margin]:
            self.log('订单取消：资金不足，代码：%s，持有资金:%i' %
                        (order.info['name'],
                         self.broker.get_cash()))

        elif order.status in [order.Rejected]:
            self.log('订单取消：被拒绝')
        
        self.inds[self.tracker]['order'] = None


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('交易收益：代码：%s, 毛利润 %.2f, 净利润 %.2f' %
                 (trade.history[0]['event']['order'].info['name'], 
                  trade.pnl, trade.pnlcomm))
        

    def next(self):

        buy_list = []
        sell_list = []

        for d in self.datas:
            # Check if an order is pending ... if yes, we cannot send a 2nd one
            if self.inds[d]['order']:
                return

            buy_s = self.inds[d]['wave_buy'][0]
            sell_s = self.inds[d]['wave_sell'][0] or self.inds[d]['bollinger_sell'][0]


            if sell_s and self.getposition(d).size > 0:
                sell_list.append((d, -1))

            elif buy_s and self.getposition(d).size == 0:
                ava_pos = self.cash_unit // (self.p.stake * d.close[0]) * self.p.stake
                buy_list.append((d, ava_pos))
        
        # Sell first
        for s_order in sell_list:
            self.tracker = s_order[0]
            self.log('卖单创建, 代码: %s, 价格: %.2f, 股数:%i' % 
                     (s_order[0]._name, s_order[0].close[0],
                     self.getposition(s_order[0]).size))
            self.inds[s_order[0]]['order'] = self.close(data=s_order[0], name=s_order[0]._name)
        
        # Buy in randomized order
        random.shuffle(buy_list)
        for b_order in buy_list:
            self.tracker = b_order[0]
            self.log('买单创建, 代码: %s, 价格: %.2f, 股数: %i' % 
                     (b_order[0]._name, b_order[0].close[0], b_order[1]))
            self.inds[b_order[0]]['order'] = self.buy(data=b_order[0], size=b_order[1], name=b_order[0]._name)

## 回测

### 个股回测

In [ ]:
stock_index = '000166.SZ'
startdate = dt.datetime(2020,1,1) - dt.timedelta(days=20)
endddate = dt.datetime(2020,12,31)


if __name__ ==  '__main__':
    
    # Initialization
    cerebro = bt.Cerebro()
    strats = cerebro.addstrategy(BW_s, printlog=True) 

    # Data
    df = utilsJ.get_stock(stock_index, startdate, endddate)
    data = btfeed.PandasData(dataname=df,fromdate=startdate,todate=endddate)
    cerebro.adddata(data)

    # Start condition
    cerebro.broker = bt.brokers.BackBroker(coc=True)   
    cerebro.broker.setcash(20000)
    #cerebro.broker.setcommission()
    start_value = cerebro.broker.getvalue()
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Execution
    cerebro.run()

    # Final result
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))
    cerebro.plot(iplot=False)

### 个股沪深300回测(调参)

In [ ]:
index_code = '399300.SZ'
startdate = dt.datetime(2020,1,1) - dt.timedelta(days=20)
enddate = dt.datetime(2020,12,31)

if __name__ == '__main__':

    index_list = utilsJ.get_index_components(index_code, startdate, enddate)
    profit_stk = dict()
    for stk in index_list:
        max_profit = (0,0.0,0)
        for i in range(3,7):
            for j in np.arange(1.5,3,0.1):

                # Initialization
                cerebro = bt.Cerebro()
                strats = cerebro.addstrategy(BW_s, printlog=False, direction_lag=i, bollinger_dev=float(j)) 

                # Data
                df = utilsJ.get_stock(stk, startdate, enddate)
                data = btfeed.PandasData(dataname=df,fromdate=startdate,todate=enddate)
                cerebro.adddata(data)

                # Start condition
                cerebro.broker = bt.brokers.BackBroker(coc=True)   
                cerebro.broker.setcash(300000)
                #cerebro.broker.setcommission()
                start_value = cerebro.broker.getvalue()
                #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

                # Execution
                cerebro.run()

                # Final result
                final_value = cerebro.broker.getvalue()
                #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
                print('%s, (%i, %.1f) Net Profit: %.2f%%' % (stk,i,j,(final_value - start_value) / start_value * 100))
                if max_profit[2] <  (final_value-start_value)/start_value*100:
                    max_profit = (i,j,(final_value-start_value)/start_value*100)
                    
        profit_stk[stk] = (final_value - start_value) / start_value * 100

### 多股回测
注：存在着Backtrader里面数据线必须对齐保持起始日期一致的问题。因此手动将数据对齐，对于晚上市的股票，手动在前面添加了空数据行。

In [ ]:
startdate = dt.datetime(2020,1,1) - dt.timedelta(days=20)
enddate = dt.datetime(2020,12,31)
date_idx =  utilsJ.get_stock('000001.SZ', startdate, enddate).index
index_code = '399300.SZ'
index_list = utilsJ.get_index_components(index_code, startdate, enddate)

if __name__ ==  '__main__':
    
    # Initialization
    cerebro = bt.Cerebro(tradehistory=True)
    strats = cerebro.addstrategy(BW_m, printlog=False, units=350)

    # Data
    for stk in index_list:
        df = utilsJ.get_stock(stk, startdate, enddate)
        df_new = pd.DataFrame([[x,0,0,0,0,0] for x in date_idx if x not in df.index], columns=['datetime', 'open', 'high', 'low', 'close', 'volume'])
        df_new.set_index('datetime', inplace=True)
        if len(df_new) != 0:
            df = pd.concat([df_new, df])
        df.sort_index(inplace=True)
        data = btfeed.PandasData(dataname=df,fromdate=startdate,todate=enddate)
        cerebro.adddata(data, name=stk)

    # Start condition
    cerebro.broker = bt.brokers.BackBroker(coc=True)
    cerebro.broker.setcash(50000000)
    #cerebro.broker.setcommission()
    start_value = cerebro.broker.getvalue()
    #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Execution
    cerebro.run(maxcpus=16)

    # Final result
    final_value = cerebro.broker.getvalue()
    #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))
    #profit.append((final_value - start_value) / start_value * 100)
    #cerebro.plot(iplot=False)

### 多股循环调参

In [ ]:
profit_dict = {}
startdate = dt.datetime(2020, 1, 1) - dt.timedelta(days=20)
enddate = dt.datetime(2020, 12, 31)
date_idx =  utilsJ.get_stock('000001.SZ', startdate, enddate).index
index_code = '399300.SZ'
index_list = utilsJ.get_index_components(index_code, startdate, enddate)


if __name__ == '__main__':

    for i in np.arange(1.5,3,0.1):

        # Initialization
        cerebro = bt.Cerebro(tradehistory=True)
        strats = cerebro.addstrategy(BW_m, printlog=False, units=len(index_list), wave_period_buy=3, wave_period_sell=6, bollinger_dev=float(i))
        
        # Data
        for stk in index_list:
            df = utilsJ.get_stock(stk, startdate, enddate)
            df_new = pd.DataFrame([[x,0,0,0,0,0] for x in date_idx if x not in df.index], columns=['datetime', 'open', 'high', 'low', 'close', 'volume'])
            df_new.set_index('datetime', inplace=True)
            if len(df_new) != 0:
                df = pd.concat([df_new, df])
            df.sort_index(inplace=True)
            data = btfeed.PandasData(dataname=df,fromdate=startdate,todate=enddate)
            cerebro.adddata(data, name=stk)
        
        # Start condition
        cerebro.broker = bt.brokers.BackBroker(coc=True)
        cerebro.broker.setcash(5000 * len(index_list) * 100)
        #cerebro.broker.setcommission()
        start_value = cerebro.broker.getvalue()
        #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

        # Execution
        cerebro.run(maxcpus=16)

        # Final result
        final_value = cerebro.broker.getvalue()
        #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
        #print('(%s, %s) Net Profit: %.2f%%' % (s[0], s[1], (final_value-start_value)/start_value*100))
        print((i, (final_value - start_value) / start_value * 100))